In [163]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing

from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

In [164]:
df = pd.read_excel("db_vf.xlsx")

In [165]:
#FUNCIÓN PARA LIMPIAR

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

def cleaning(text):
    text = text.lower()
    text = normalize(text)
    doc = nlp(text)
    return doc

In [166]:
dfe =  df.copy()
dfe.rename(columns = {"educ_history_clean": "education"}, inplace = True)

dfe["education"] =  dfe["education"].str.lower()
# dfe = dfe[dfe["etapa"] == 1]

In [167]:
L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]


K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            try:
                c["start"] = int(c["start"])
            except:
                None

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            try:
                c["end"] = int(c["end"])
            except:
                None

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

dfe["educacion"] = K

In [168]:
dfe = dfe.reset_index(drop = True)

In [169]:
#SELECCIONAR A LOS UNIVERSITARIOS SOLO SI ESTUDIARON MÁS DE 2 AÑOS
dfe["max_univ"] = 0

for i in tqdm(range(len(dfe["educacion"]))):
    j = [0]

    for e in dfe["educacion"][i]:

        if "start" in e.keys() and "end" in e.keys() and type(e["start"]) == int and type(e["end"]) == int:
            resta = e["end"] - e["start"]
            j.append(resta)
               
    p = max(j)
    try:
        dfe["max_univ"][i] = p
    except:
        None

  0%|          | 0/1004 [00:00<?, ?it/s]<ipython-input-169-b1d6b2bb3482>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["max_univ"][i] = p
100%|██████████| 1004/1004 [00:00<00:00, 8296.81it/s]


In [170]:
for i in dfe["educacion"]:
    for e in i:
        if "degree" in e.keys():
            e["degrees"] = cleaning(e["degrees"])

In [171]:
sp = ["specialisation", "specialization", "especialização", "especialización", "especialista"]
ms = ["master", "msc ", "m.a. ", "m.s. ", "mba ", "mphill ", "mres ", "mag ", " msc", " m.a.", " m.s.", " mba", " mphill", " mres", " mag", "maestria", "magister", "maestría"]
ph = ["phd", "ph.d." "dr ", "dr phil", "doctor", "doctorado", "doutor", "doutorado", "ph.d"]


#FUNCION PARA ENCONTRAR GRADO
def degree(palabra):
    P = [1]

    for t in sp:
        if t in palabra:
            P.append(2)

    for k in ms:
        if k in palabra:
            P.append(3)

    for e in ph:
        if e in palabra:
            P.append(4)
    
    if len(P)>0:
        return max(P)

In [172]:
# dfe.groupby(["max_univ"]).count()
dfe["max_univ"] = dfe["max_univ"].replace(np.nan, 0)

In [173]:
dfe["degree"] = np.nan
dfe["degree"] = dfe["max_univ"].apply(lambda x: 0 if x<=1 else 1) #si tiene más de 3 años es por lo menos bachelor

In [174]:
#crear columna de titulo de grado
dfe["educ_title"] = np.nan

In [175]:

#extrar titulo de grado
for i in range(len(dfe)):
    for e in dfe["educacion"][i]:
        if "start" in e.keys() and "end" in e.keys() and "degrees" in e.keys() and type(e["start"]) == int and type(e["end"]) == int:
            resta = e["end"] - e["start"]
            if resta>=0:
                dfe["educ_title"][i] =e["degrees"]

            if resta>=1:
                dfe["educ_title"][i] =e["degrees"]

            if resta>=2:
                dfe["educ_title"][i] =e["degrees"]

            if resta>=3:
                dfe["educ_title"][i] =e["degrees"]
        if "degrees" in e.keys():
            dfe["educ_title"][i] =e["degrees"]



<ipython-input-175-b15059e5266c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["educ_title"][i] =e["degrees"]
C:\Users\War\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-175-b15059e5266c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["educ_title"][i] =e["degrees"]
<ipython-input-175-b15059e5266c>:10: SettingW

In [176]:
#titulo de grado y alcance
for i in range(len(dfe)):
    L = []
    q = []
    if  dfe["max_univ"][i]>0:
        for e in dfe["educacion"][i]: #dejo entrar a los que por lo menso tienen 1 año (para que entren los que tengan maestría), quizas hubo alguien que coloco solo la maestria de un año
            if "degrees" in e and len(e["degrees"])>0:
                L.append(degree(e["degrees"]))
                q.append(e["degrees"])

        if len(L)>0 and max(L)>1 :
            dfe["degree"][i] = max(L)
            dfe["educ_title"][i] = q[L.index(max(L))]
             


<ipython-input-176-d841080eaac5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["degree"][i] = max(L)
<ipython-input-176-d841080eaac5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["educ_title"][i] = q[L.index(max(L))]


In [178]:
dfe.to_excel("final_DB.xlsx", index = False)

In [ ]:
dfb